<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/e/ed/Logo_Universit%C3%A9_du_Maine.svg/1280px-Logo_Universit%C3%A9_du_Maine.svg.png' width="300" height="500">
<br>
<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Infrastructures pour l'IA</h1>
    <p style='text-align: center;'>Master Informatique 1</p>
    <p style='text-align: center;'>Anhony Larcher</p>
</div>

Cette session est organisée comme un challenge:
* Vous optimiserez un code afin d'effectuer un calcul le plus rapidement possible
* le résultat sera utilisée dans le TP numéro 6 pour faire du calcul parallélisé.


# Implémenter le calcul suivant et optimisez le au mieux

Il s'agit d'accumuler les statistiques d'ordre 0 et d'ordre 1 sur un mélange de Gaussiennes. Vous trouverez sur Umtice une archive zip contenant un objet Mixture et des paramètres acoustiques de type MFCC.

Chaque distribution de la mixture a pour densité de probabilité:

Et la densité de probabilité du mélange de Gaussienne est 
$$p(x|\lambda) = \Sigma_{i=1}^{M}w_ip_i(x)$$

où $$\Sigma_{i=1}^{M}w_i = 1$$

et 

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu)^T\Sigma_i^{-1}(x-\mu))}$$

Nous souhaiton ici calculer pour chaque trame $x_t$

$$P(i|x_t) = \frac{w_ip_i(x_t)}{\Sigma_{j=1}^M w_j p_j(x(t))}$$

Pour ensuite calculer les statistiques d'ordre 0 pour chaque gaussienne $i$:

$$n_i = \Sigma_{t=1}^{T}P(i|x_t)$$

Les statistiques d'ordre 1:

$$E_i(x) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t$$

Et les statistiques d'ordre 2:

$$E_i(x^2) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t^2$$

Les données sont disponibles ici:

    https://umbox.univ-lemans.fr/index.php/s/CiP47cfw8NBMM5J

## 1.1) Initialisez l'accumulateur de statistiques

Le fichier ``mixture.py``contient le code d'une classe Mixtuyre qui sera utilisé dans ce TP. Ce code se trouve également dans la cellule ci-dessous pour travailler dans le notebook. Une fois la cellule ci-dessous complétée vous pourrez recopier le code dans le fichier ``mixture.py``afin d'avoir une version propre du code.

In [30]:
import numpy
import pickle


class Mixture(object):

    def __init__(self):
        """
        Initialize an empty Mixture object

        self.mu est de dimension 64x39
        self.invcov est de dimension 64x39
        numpy.square(self.mu) * self.invcov) est de dimension 64x39
        numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
        A est de dimension 64
        """
        
        self.w = numpy.array([]) 
        self.mu = numpy.array([]) 
        self.invcov = numpy.array([]) 
        self.cst = numpy.array([])
        self.det = numpy.array([])
        self.dim = numpy.shape

    def __repr__(self):
        """
        Serialize a Mixture object to text
        """
        return f'w = {self.w}\nmu =  {selmf.mu}\ninvcov = {self.invcov}, cst = {self.cst}\ndet = {self.det}'

    @classmethod
    def read(cls, filename):
        """
        Read a Mixture object stored in Pickle format on disk
        :param filename: the name of the file to read from
        :return: a Mixture object
        """
        with open(filename, 'rb') as fh:
            mixture = pickle.load(fh)
            return mixture

    def save(self, filename):
        """
        Save a Mixture object to disk in Pickle format
        :param filename: the name of the file to write to
        """
        with open(filename, 'wb') as fh:
            pickle.dump(self, fh)


In [31]:
import numpy as np
data=np.load("../data_1/features_1.npy")
data.shape #338 est le nombre de vecteurs dans le fichier et 39 est la dimension de chaque vecteur

(338, 39)

In [35]:
# CRéation d'un objet mixture

self= Mixture()
self.w = np.load("w.npy")
self.mu = np.load("mu.npy")
self.invcov = np.load("invcov.npy")
self.dim = self.invcov.shape

#features = données

## Analyse des formules

On remarque d'abord que certains éléments dépendent des données, et plus particulièrement de chaque vecteur de donnée, mais également de chaque distribution de la mixture.

Nous verrons donc apparaitre 2 boucles principales:
* une sur les données
* une sur les distributions

Pour simplifier le calcul nous devons donc séparer ce qui est indépendant de ces boucles afin de ne pas le recalculer plusieurs fois.

Ré-écrivez $$p_i(x)$$ en séparant ces termes.

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i))}$$

On trouve d'abord un terme qui dépend de chaque mixture: leur déterminant

In [33]:
# Il faut donc calculer ce déterminant une seule fois:
# Le déterminant d'une matrice diagonale est le produit de ses termes diagonaux.
# Les covariances inverses étant stockées par ligne dans self.invcov nous calculons

self.det = np.zeros(self.dim)
self.det = 1.0 / np.prod(self.invcov, axis=1)

# chaque x de la formule vient des features et les mu viennet du mu de mixture 


Nous pouvons maintenant calculer le premier terme pour chaque distribution:
    
$$ \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} $$ 

In [49]:
# Pour chaque distribution, le terme invariant est:

for ii in range(39):
    self.cst[ii] = 1.0 / (numpy.sqrt(self.det[ii]) * (2.0 * numpy.pi) ** (39/2.0))

# il faut calculer les termes qui ne dépendent pas de x.


IndexError: index 0 is out of bounds for axis 0 with size 0

Regardons maintenant le terme contenu dans l'exponentielle:

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) $$ 

Tout ce terme dépend des distributions, une partie seulement dépend des données.

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 

Calculez chacun de ces termes:

Analyse du terme independant des données:

$$ \mu_i^T\Sigma_i^{-1}\mu_i $$

$$ \mu_i^T$$ est de dimension (1, 39) $$\Sigma_i$$ est de dimension (39, 39)

Donc $$ \mu_i^T\Sigma_i^{-1}\mu_i $$ est de dimension (1,)

In [48]:
# En pratique, comme la matrice de covariance est diagonale, nous n'avons pas besoin de calculer un vrai produit matriciel
# Écrivez la valeur du premier terme de \mu_i \Sigma^{-1}_i et tirez partie de cette expression pour simplifier
# le calcul de ce terme

# self.mu est de dimension 64x39
# self.invcov est de dimension 64x39
# numpy.square(self.mu) * self.invcov) est de dimension 64x39
# numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
# A est de dimension 64

for j in range(64):
    for i in range(39):
        tmp = np.log(self.cst[j]) + (data[i]*data[i]*self.invcov[j]) - 2(self.mu[j]*self.invcov[j]*data[i]) + (self.mu[j]*self.invcov[j])
        print(tmp)

<>:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
/var/folders/ts/kww6z2z510xb35njpx1_8q8m0000gn/T/ipykernel_74093/961328292.py:13: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  tmp = np.log(self.cst[j]) + (data[i]*data[i]*self.invcov[j]) - 2(self.mu[j]*self.invcov[j]*data[i]) + (self.mu[j]*self.invcov[j])


IndexError: index 0 is out of bounds for axis 0 with size 0

## 1.2) Écrivez une fonction compute_lpp 

Cette méthode de la class **Mixture** va calculer les log posterior probabilités $$\log{p_i(x)}$$ d'un ensemble de vecteurs sur ce mélange de Gaussienne à matrices de covariances diagonales

Pour acumuler les statistiques, vous allez créer un objet mixture que vous appelerez **accum**.

* Cette méthode prend en paramètres une matrice de coefficients cepstraux de dimension N x F où N est le nombre de vecteurs (variable selon les fichiers) et F est la dimension des vecteurs (39 dans notre cas)
* En premier lieu, pensez à extraire des boucles tous les termes qui ne dépendent pas des données
* Cette méthode renvoie les $$n_i$$ (donc un vecteur)

In [2]:
import numpy
X = numpy.load("features_1.npy")
X.shape

(338, 39)

## 1.3) Utilisez la fonction **sum_log_probabilities**
* Que fait cette fonction qui vous est donnée?
* Pourquoi l'utilise-t'on?

In [2]:
def sum_log_probabilities(lp):
    """Sum log probabilities in a secure manner to avoid extreme values

    :param lp: numpy array of log-probabilities to sum
    """
    pp_max = numpy.max(lp, axis=1)
    log_lk = pp_max + numpy.log(numpy.sum(numpy.exp((lp.transpose() - pp_max).T), axis=1))
    ind = ~numpy.isfinite(pp_max)
    if sum(ind) != 0:
        log_lk[ind] = pp_max[ind]
    pp = numpy.exp((lp.transpose() - log_lk).transpose())
    llk = log_lk.sum()
    return pp, llk

## 1.4) Bouclez sur les fichiers de paramètres pour accumuler les statistiques